Code from the Kernel describing how to build a basic CNN model. Model taken from Datacamp tutorial.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
from zipfile import ZipFile
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [ ]:
print(os.listdir("../input"))

In [ ]:
dataset = [] #load image data here
labels = [] #load the labels here from training data

In [ ]:
train_Data = pd.read_csv("../input/train.csv") #load train data here from CSV file
#train_Data.head() #confirm by viewing the first 5 rows
id_code_Data = train_Data.id_code
diagnosis_Data = train_Data.diagnosis

In [ ]:
#sampling data to plot and take a closer look at the images
temp_trainData = train_Data
temp_trainData.sort_values(by=['diagnosis'])
sample_trainData = temp_trainData.sample(50)
sample_trainData = sample_trainData.sort_values(by=['diagnosis'])
sample_ids = sample_trainData['id_code']
sample_ids = sample_ids.reset_index(drop=True)
sample_diagnosis = sample_trainData['diagnosis']
sample_diagnosis = sample_diagnosis.reset_index(drop=True)

In [ ]:
'''import numpy as np
import matplotlib.pyplot as plt

w=10
h=10
fig=plt.figure(figsize=(8, 8))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    img = np.random.randint(10, size=(h,w))
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()'''
from matplotlib import image
fig = plt.figure(figsize=(50, 50))
columns = 5
rows = 10
for i in range(1, columns*rows+1):
    path = sample_ids[i-1] + '.png'
    img = image.imread('../input/train_images/' + path)
    #data = image.imread('../input/train_images/' + img)
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
    #plt.xlabel(sample_diagnosis[i-1])
    plt.title(sample_diagnosis[i-1], fontsize=20)
plt.show()

In [ ]:
#define the function to prepare the images
def prepare_Images(label,path):
    img=cv2.imread(path,cv2.IMREAD_COLOR)
    img_res=cv2.resize(img,(100,100))
    img_array = img_to_array(img_res)
    img_array = img_array/255
    dataset.append(img_array)
    labels.append(str(label))

In [ ]:
#for id_code,diagnosis in tqdm(zip(id_code_Data,diagnosis_Data)):
 #   path = os.path.join('../input/train_images','{}.png'.format(id_code))
    

In [ ]:
for id_code,diagnosis in tqdm(zip(id_code_Data,diagnosis_Data)):
    path = os.path.join('../input/train_images','{}.png'.format(id_code))
    prepare_Images(diagnosis,path)
    #print (id_code, diagnosis)

In [ ]:
#Convert list to numpy array
images = np.array(dataset)
label_arr = np.array(labels)
#label_arr

In [ ]:
#spliting the training data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(images,label_arr,test_size=0.20,random_state=42)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#Convert to class labels categorical
y_train = np_utils.to_categorical(y_train, num_classes=5)
y_test = np_utils.to_categorical(y_test, num_classes=5)

In [ ]:
#Building the model
model = Sequential()
# First convolutional layer accepts image input
model.add(Conv2D(filters=32, kernel_size=5, padding='same', activation='relu', 
                        input_shape=(100, 100, 3)))
# Add a max pooling layer
model.add(MaxPooling2D(4))
# Add a convolutional layer
model.add(Conv2D(filters=64, kernel_size=5, padding='same', activation='relu'))
# Add another max pooling layer
model.add(MaxPooling2D(4))
model.add(Conv2D(filters=64, kernel_size=5, padding='same', activation='relu'))
# Add another max pooling layer
# Flatten and feed to output layer
model.add(Flatten())
model.add(Dense(5, activation='softmax'))

# Summarize the model
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#hist = model.fit(x_train,y_train,batch_size=64,epochs=10,validation_data=(x_test, y_test))
#model.fit(x_train,y_train,batch_size=64,epochs=10)
#pred = model.predict(x_test)
#score = model.evaluate(x_test, y_test, batch_size=16)

In [ ]:
hist = model.fit(x_train, y_train, validation_split=.2, epochs=10)

In [ ]:
# Obtain accuracy on test set
score = model.evaluate(x=x_test, 
                       y=y_test,
                       verbose=0)
print('Test accuracy:', score[1])

In [ ]:
pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
acc_score = round(accuracy_score(y_test.argmax(axis=1), pred.argmax(axis=1)),2)
print(acc_score)
report = classification_report(y_test.argmax(axis=1), pred.argmax(axis=1))
print(report)
conMat = confusion_matrix(y_test.argmax(axis=1),pred.argmax(axis=1))
print(conMat)

In [ ]:
# Any results you write to the current directory are saved as output.
test_df = pd.read_csv('../input/test.csv')
test_df.head()

In [ ]:
x = test_df['id_code']
test_Dataset = []

In [ ]:
def make_test_data(path):
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    img_res = cv2.resize(img, (100,100))
    img_array = img_to_array(img_res)
    img_array = img_array/255
    test_Dataset.append(img_array)

In [ ]:
for id_code in tqdm(x):
    path = os.path.join('../input/test_images','{}.png'.format(id_code))
    make_test_data(path)

In [ ]:
test_image = np.array(test_Dataset) #convert the test image to np array

In [ ]:
pred=model.predict(test_image)

In [ ]:
pred.shape

In [ ]:
pred=np.argmax(pred,axis=1)

In [ ]:
sub_df = pd.read_csv('../input/sample_submission.csv')
sub_df.head()

In [ ]:
sub_df.diagnosis = pred
sub_df.head()

In [ ]:
sub_df.to_csv("submission.csv",index=False)